In [133]:
# this might be needed if you don't have such library
# in case of any issues remove capture magic, since
# it hides output of the cell
%%capture
!pip3 install audiomentations

In [134]:
from audiomentations import Compose, AddGaussianNoise, Shift, Normalize, TanhDistortion, Gain
import matplotlib.pyplot as plt
import os
import scipy as sp

In [135]:
SAMPLES_DIR = './barbie_vs_puppy/'
TRAINING_DIR = './training/'
VALIDATION_DIR = './validation/'
N_AUGMENTATIONS = 10

In [136]:
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TanhDistortion(p=0.5),
    Gain(p=0.5),
    Shift(min_fraction=-0.25, max_fraction=0.25, p=0.5),
    Normalize(p=1.0),
])

In [137]:
# save audio spectrogram to the corresponding directory
# here i use 20% of data for validation
def save_spct(audio, sr, suffix, cnt):
    f, t, Sxx = sp.signal.spectrogram(audio, sr)
    fig = plt.figure()
    plt.pcolormesh(t, f, Sxx, shading='gouraud')
    plt.axis('off')
    plt.ylim([0, 10000])
    plt.xlim([0, 2])
    if cnt % 10 >= 8:
        plt.savefig(VALIDATION_DIR + '/' + suffix + '/' + str(cnt) + '.png')
    else:
        plt.savefig(TRAINING_DIR + '/' + suffix + '/' + str(cnt) + '.png')
    plt.close(fig)

In [138]:
# read sequentially audio files and
# pass them to the augment function
# after that save the spectrogram
cnt = 0
for suffix in ['barbie', 'puppy']:
    for file in os.listdir(SAMPLES_DIR + suffix):
        if file.endswith('.wav'):
            sr, audio = sp.io.wavfile.read(os.path.join(SAMPLES_DIR + suffix, file))
            if len(audio.shape) > 1 and audio.shape[1] > 1:
                audio = audio[:, 0]

            save_spct(audio, sr, suffix, cnt)
            cnt += 1
            for i in range(N_AUGMENTATIONS):
                audio = augment(audio, sr)
                save_spct(audio, sr, suffix, cnt)
                cnt += 1

/Users/m8dotpie/opt/anaconda3/lib/python3.9/site-packages/audiomentations/core/transforms_interface.py:57: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
